In [52]:
import pygame
import math
from pygame.sprite import Group
#import U as Map
pygame.init()
bullets = Group()
size = (800, 800)
clock = pygame.time.Clock()
image = pygame.image.load("Tankx1.png")

In [53]:
imageb11 = pygame.image.load('Bullet11.png')
imageb12 = pygame.image.load('Bullet12.png')
imageb13 = pygame.image.load('Bullet13.png')
imageb14 = pygame.image.load('Bullet14.png')
imageb15 = pygame.image.load('Bullet15.png')
imageb21 = pygame.image.load('Bullet21.png')
imageb22 = pygame.image.load('Bullet22.png')
imageb23 = pygame.image.load('Bullet23.png')
imageb24 = pygame.image.load('Bullet24.png')
imageb25 = pygame.image.load('Bullet25.png')

In [54]:
def check_event_rotate(event, angle):
    if event.type == pygame.KEYDOWN:
        if event.key == pygame.K_d:
            angle += 1
        if event.key == pygame.K_a:
            angle -= 1
    return angle

In [55]:
def check_event_velocity(event, x, y, vx, vy, angle):
    if event.type == pygame.KEYDOWN:
        if event.key == pygame.K_w:
            x += vx * math.cos(angle)
            y += vy * math.sin(angle)
    return (x, y)

In [56]:
class Tank(pygame.sprite.Sprite):
    def __init__(self, x, y, image, vx, vy):
        pygame.sprite.Sprite.__init__(self)
        self.image_non_actual = image
        self.rect = self.image_non_actual.get_rect(center=(x, y))
        colorkey = self.image_non_actual.get_at((0, 0))
        self.image_non_actual.set_colorkey(colorkey)
        self.image_actual = self.image_non_actual
        self.vx = vx
        self.vy = vy
        self.angle = 0
        self.vx2 = 0
        self.vy2 = 0
        self.angle2 = 0
        self.rx = 45
        self.ry = 26
        
    def update_velocity(self, event):
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_w:
                self.vx2 = self.vx * math.cos(-1 * math.radians(self.angle))
                self.vy2 = self.vy * math.sin(-1 * math.radians(self.angle))
            if event.key == pygame.K_s:
                self.vx2 = -1 * self.vx * math.cos(-1 * math.radians(self.angle))
                self.vy2 = -1 * self.vy * math.sin(-1 * math.radians(self.angle))
            if event.key == pygame.K_d:
                self.angle2 = -2
            if event.key == pygame.K_a:
                self.angle2 = +2
        if event.type == pygame.KEYUP:
            if event.key in [pygame.K_w, pygame.K_s]:
                self.vx2 = 0
                self.vy2 = 0
            if event.key in [pygame.K_a, pygame.K_d]:
                self.angle2 = 0
        self.angle += self.angle2
        self.rect.x += self.vx2
        self.rect.y += self.vy2
        
    def update_rotate(self, event):
        self.image_actual = pygame.transform.rotate(self.image_non_actual, self.angle)
    def bounce(self, X, Y, x0, y0):
        if self.rect.x + self.image_actual.get_size()[0] >= X:
            self.rect.x = X - self.image_actual.get_size()[0]
        if self.rect.x <= x0:
            self.rect.x = x0
        if self.rect.y + self.image_actual.get_size()[1] >= Y:
            self.rect.y = Y - self.image_actual.get_size()[1]
        if self.rect.y <= y0:
            self.rect.y = y0
    def draw_self(self):
        screen.blit(self.image_actual, self.rect.topleft)
        #pygame.draw.rect(screen, (255, 0, 0), (self.rect.x, self.rect.y, self.image_actual.get_size()[0], self.image_actual.get_size()[1]), 1)

In [57]:
def fire(bullets, tank, event, image):
    mouse = pygame.mouse.get_pressed()[0]
    if event.type == pygame.KEYDOWN:
        if event.key == pygame.K_SPACE:
            x = tank.rect.x + tank.image_actual.get_rect().center[0] + tank.rx * (math.cos(-1 * math.radians(tank.angle))) + 5.5
            y = tank.rect.y + tank.image_actual.get_rect().center[1] + tank.rx * (math.sin(-1 * math.radians(tank.angle)))
            bullet = Bullet(x, y, image, 5, 5, tank.angle)
            bullets.add(bullet)
    return bullets

In [58]:
class Bullet(pygame.sprite.Sprite):
    def __init__(self, x, y, image, vx, vy, angle):
        pygame.sprite.Sprite.__init__(self)
        self.image = image
        self.rect = self.image.get_rect(center=(x, y))
        self.vx = vx
        self.vy = vy
        self.angle = angle
        self.counter = 0
        self.hits = 0
        self.rx = 20
        
    def update1(self, imageb11, imageb12, imageb13, imageb14, imageb15):
        if self.counter % 15 == 0:
            self.image = imageb11
            self.image = pygame.transform.rotate(self.image, self.angle)
        elif self.counter % 15 == 3:
            self.image = imageb12
            self.image = pygame.transform.rotate(self.image, self.angle)
        elif self.counter % 15 == 6:
            self.image = imageb13
            self.image = pygame.transform.rotate(self.image, self.angle)
        elif self.counter % 15 == 9:
            self.image = imageb14
            self.image = pygame.transform.rotate(self.image, self.angle)
        elif self.counter % 15 == 12:
            self.image = imageb15
            self.image = pygame.transform.rotate(self.image, self.angle)
        self.counter += 1
        self.rect.x += int(self.vx * math.cos(math.radians(-1 * self.angle)))
        self.rect.y += int(self.vy * math.sin(math.radians(-1 * self.angle)))
    def bounce(self, X, Y):
        if self.rect.x + self.rx >= X:
            self.rect.x = X - self.rx
            self.hits += 1
            self.angle = 180 - self.angle
        if self.rect.x <= 0:
            self.rect.x = 0
            self.hits += 1
            self.angle = 180 - self.angle
        if self.rect.y >= Y:
            self.rect.y = Y
            self.hits += 1
            self.angle = - self.angle
        if self.rect.y <= 0:
            self.rect.y = 0
            self.hits += 1
            self.angle = - self.angle
    def draw_self(self):
        screen.blit(self.image, self.rect)

In [59]:
def Hit(map1, tank, size, q):
    for i in range(size[1] // q[1]):
        for x in range(size[0] // q[0]):
            if map1.map[i][x] == 1:
                if tank.rect.y + tank.image_actual.get_size()[1] // 2 <= (i + 1) * map1.cell_y and tank.rect.y  + tank.image_actual.get_size()[1] // 2 >= i * map1.cell_y:
                    pygame.draw.circle(screen, (100, 155, 155), (0, 0), 5)
                    if tank.rect.x + tank.image_actual.get_size()[0] >= x * map1.cell_x and tank.rect.x <= x * map1.cell_x:
                        tank.rect.x = x * map1.cell_x - tank.image_actual.get_size()[0]
                    if tank.rect.x <= (x + 1) * map1.cell_x and tank.rect.x + tank.image_actual.get_size()[0] >= (x + 1) * map1.cell_x:
                        tank.rect.x = (x + 1) * map1.cell_x
                else:
                    if tank.rect.x + tank.image_actual.get_size()[0] // 2 > x * map1.cell_x and tank.rect.x + tank.image_actual.get_size()[0] // 2 < (x + 1) * map1.cell_x:
                        if tank.rect.y <= (i + 1) * map1.cell_y and tank.rect.y + tank.image_actual.get_size()[0] >= (i + 1) * map1.cell_y:
                            tank.rect.y = (i + 1) * map1.cell_y
                        elif tank.rect.y <= (i) * map1.cell_y and tank.rect.y + tank.image_actual.get_size()[1] >= (i) * map1.cell_y:
                            tank.rect.y = (i) * map1.cell_y - tank.image_actual.get_size()[1]
    return tank 

In [60]:
def Check(bullets, Map, size, q):
    for i in range(size[1] // q[1]):
        for x in range(size[0] // q[0]):
            for bullet in bullets.sprites():
                if bullet.rect.y >= i * Map.cell_y and bullet.rect.y <= (i + 1) * Map.cell_y:
                    if bullet.rect.x >= x * Map.cell_x and bullet.rect.x <= (x + 1) * Map.cell_x:
                        if Map.map[i][x] == 1:
                            bullets.remove(bullet)
                            Map.map[i][x] = 2
    return (bullets, Map)

In [61]:
class Map(pygame.sprite.Sprite):
    def __init__(self, size, q):
        self.cell_x = q[0]
        self.cell_y = q[1]
        self.map = []
        self.counter = 0
        for i in range(size[1] // q[1]):
            self.map.append([])
            for x in range(size[0] // q[0]):
                #self.map[i].append(randint(0, 1))
                self.map[i].append(0)
    def draw_map(self, size, q, screen, image):
        for i in range(size[1] // q[1]):
            for x in range(size[0] // q[0]):
                if self.map[i][x] == 1:
                    pygame.draw.rect(screen, (100, 56, 179), (x * self.cell_x, i * self.cell_y, self.cell_x, self.cell_y))
                    pygame.draw.rect(screen, (0, 0, 0), (x * self.cell_x, i * self.cell_y, self.cell_x, self.cell_y), 1)
                if self.map[i][x] == 0:
                    pygame.draw.rect(screen, (200, 156, 179), (x * self.cell_x, i * self.cell_y, self.cell_x, self.cell_y))
                    pygame.draw.rect(screen, (0, 0, 0), (x * self.cell_x, i * self.cell_y, self.cell_x, self.cell_y), 1)
                #if self.map[i][x] > 1:
                for k in range(2, 27, 3):
                    if self.map[i][x] == k:
                        pygame.draw.rect(screen, (200, 156, 179), (x * self.cell_x, i * self.cell_y, self.cell_x, self.cell_y))
                        pygame.draw.rect(screen, (0, 0, 0), (x * self.cell_x, i * self.cell_y, self.cell_x, self.cell_y), 1)
                        screen.blit(image[(k - 2) // 3], (x * self.cell_x, i * self.cell_y))
                    #pygame.draw.rect(screen, (0, 210, 50), (x * self.cell_x, i * self.cell_y, self.cell_x, self.cell_y))
                
                if self.map[i][x] > 1:
                    self.map[i][x] += 1
                self.map[i][x] = self.map[i][x] % 26

In [62]:
def click(Map, event, size, q, i, x):
    #click = pygame.mouse.get_pressed()[0]
    #pos = pygame.mouse.get_pos()
    click = 0
    if event.type == pygame.KEYDOWN:
        if event.key == pygame.K_RIGHT:
            if x < size[0]: x += 1
        if event.key == pygame.K_LEFT:
            if x > 0: x -= 1
        if event.key == pygame.K_DOWN:
            if i < size[1]: i += 1
        if event.key == pygame.K_UP:
            if i > 0: i -= 1
        if event.key == pygame.K_RETURN:
            click = True
    pos = (x, i)
    if click:
        Map.map[i][x] = (Map.map[i][x] + 1) % 2 
    
    return (Map, i, x)

In [63]:
def check(bullets, map):
    for i in range(size[1] // q[1]):
        for x in range(size[0] // q[0]):
            for bullet in bullets.sprites():
                if bullet.rect.y >= i * map.cell_y and bullet.rect.y >= (i + 1) * map.cell_y:
                    if bullet.rect.x >= x * map.cell_x and bullet.rect.x >= (x + 1) * map.cell_x:
                        bullets.remove(bullet)
                        map.map[i][x] = 2
    return (bullets, map)

In [64]:
x = 300
y = 300
vx = 5
vy = 5
x1 = 0
y1 = 0
c = 0
k = True
angle = 10
tank = Tank(x, y, image, vx, vy)

In [65]:
image1 = pygame.image.load('Expload1.png')
image2 = pygame.image.load('Expload2.png')
image3 = pygame.image.load('Expload3.png')
image4 = pygame.image.load('Expload4.png')
image5 = pygame.image.load('Expload5.png')
image6 = pygame.image.load('Expload6.png')
image7 = pygame.image.load('Expload7.png')
image8 = pygame.image.load('Expload8.png')
image = [image1, image2, image3, image4, image5, image6, image7, image8]
import random
from random import randint
clock = pygame.time.Clock()
size = (800, 800)
screen = pygame.display.set_mode(size)
k = True
q = (40, 40)
map1 = Map(size, q)
(i0, x0) = (0, 0)
while k:
    clock.tick(10)
    map1.draw_map(size, q, screen, image)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            k = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                k = False
        (map1, i0, x0) = click(map1, event, size, q, i0, x0)
    pygame.draw.rect(screen, (255, 255, 255), (x0 * map1.cell_x, i0 * map1.cell_y, map1.cell_x, map1.cell_y), 3)
    pygame.display.flip()
    screen.fill(0)

In [66]:

k = True

In [67]:
while k:
    clock.tick(60)
    map1.draw_map(size, q, screen, image)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            k = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                k = False
        bullets = fire(bullets, tank, event, imageb11)
        
    tank.update_rotate(event)
    tank.update_velocity(event)
    tank.bounce(size[0], size[1], 0, 0)
    tank.draw_self()
    tank = Hit(map1, tank, size, q)
    for bullet in bullets.sprites():
        bullet.update1(imageb11, imageb12, imageb13, imageb14, imageb15)
        bullet.draw_self()
        bullet.bounce(size[0], size[1])
        if bullet.hits >= 3:
            bullets.remove(bullet)
    
    (bullets, map1) = Check(bullets, map1, size, q)
    pygame.display.update()
    #screen.fill(0)
    bullets.update()
pygame.quit()

In [68]:
pygame.quit()